In [19]:
%matplotlib inline
import numpy as np
import math
import random
import pyJHTDB
import matplotlib.pyplot as plt
import pandas as pd

In [31]:
data = pyJHTDB.dbinfo.channel
data

{'name': 'channel',
 'xnodes': array([0.0000000e+00, 1.2271847e-02, 2.4543693e-02, ..., 2.5095926e+01,
        2.5108198e+01, 2.5120470e+01], dtype=float32),
 'ynodes': array([-1.        , -0.9999835 , -0.999945  , -0.9998791 , -0.99978054,
        -0.9996437 , -0.99946326, -0.9992338 , -0.99896646, -0.9986612 ,
        -0.9983181 , -0.9979371 , -0.99751824, -0.99706155, -0.9965671 ,
        -0.99603474, -0.9954647 , -0.99485683, -0.99421126, -0.99352795,
        -0.99280703, -0.9920484 , -0.9912521 , -0.99041826, -0.98954684,
        -0.98863786, -0.98769146, -0.9867075 , -0.9856862 , -0.9846274 ,
        -0.98353136, -0.982398  , -0.98122734, -0.98001945, -0.9787744 ,
        -0.9774923 , -0.97617304, -0.97481674, -0.97342354, -0.9719934 ,
        -0.97052634, -0.9690225 , -0.9674819 , -0.9659046 , -0.9642907 ,
        -0.96264017, -0.9609531 , -0.95922965, -0.9574698 , -0.95567364,
        -0.9538412 , -0.9519726 , -0.95006794, -0.94812715, -0.9461505 ,
        -0.9441379 , -0.94208

In [37]:
data['xnodes'], data['ynodes'], data['znodes'], data['time']

(array([0.0000000e+00, 1.2271847e-02, 2.4543693e-02, ..., 2.5095926e+01,
        2.5108198e+01, 2.5120470e+01], dtype=float32),
 array([-1.        , -0.9999835 , -0.999945  , -0.9998791 , -0.99978054,
        -0.9996437 , -0.99946326, -0.9992338 , -0.99896646, -0.9986612 ,
        -0.9983181 , -0.9979371 , -0.99751824, -0.99706155, -0.9965671 ,
        -0.99603474, -0.9954647 , -0.99485683, -0.99421126, -0.99352795,
        -0.99280703, -0.9920484 , -0.9912521 , -0.99041826, -0.98954684,
        -0.98863786, -0.98769146, -0.9867075 , -0.9856862 , -0.9846274 ,
        -0.98353136, -0.982398  , -0.98122734, -0.98001945, -0.9787744 ,
        -0.9774923 , -0.97617304, -0.97481674, -0.97342354, -0.9719934 ,
        -0.97052634, -0.9690225 , -0.9674819 , -0.9659046 , -0.9642907 ,
        -0.96264017, -0.9609531 , -0.95922965, -0.9574698 , -0.95567364,
        -0.9538412 , -0.9519726 , -0.95006794, -0.94812715, -0.9461505 ,
        -0.9441379 , -0.94208944, -0.9400053 , -0.9378855 , -0.935730

In [38]:
np.shape( data['xnodes'] ), np.shape( data['ynodes'] ), np.shape( data['znodes'] ), np.shape( data['time'] )

((2048,), (512,), (1536,), (4000,))

In [40]:
xnodes = data['xnodes']; ynodes = data['ynodes']; znodes = data['znodes']; tnodes = data['time']

In [41]:
N = 3
nx = np.shape( data['xnodes'] )
ny = np.shape( data['ynodes'] )
nz = np.shape( data['znodes'] )
nt = np.shape( data['time'] )

In [3]:
spatialInterp  = 6  # 6 point Lagrange
temporalInterp = 0  # no time interpolation
FD4Lag4        = 44 # 4 point Lagrange interp for derivatives

# mhdc has starttime .364 and endtime .376
startTime = time
endTime = startTime + 0.012
lag_dt = 0.0004

In [4]:
random.uniform(0, 8*3.14)

23.88085717545815

In [15]:
# Select points in the database to query
lpoints = []
for i in range(0,N):
    lpoints.append( [random.uniform(0, 8*3.14), random.uniform(-1, 1), random.uniform(0, 3*3.14)] )

# 2D array with double precision values
points = np.array( lpoints, dtype='float32')
points, np.shape(points)

(array([[11.349172  , -0.34695953,  6.1366253 ],
        [ 5.752386  , -0.56792283,  9.062306  ],
        [15.818903  ,  0.11588102,  6.1939235 ]], dtype=float32), (3, 3))

In [16]:
# load shared library
lTDB = pyJHTDB.libJHTDB()

# initialize webservices
lTDB.initialize()

# add token
auth_token  = "hu.elte.caesar.abiricz-fe734327"
lTDB.add_token(auth_token)

print('Coordinates of {0} points where variables are requested:'.format(N))
for p in range(N):
    print('{0}: {1}'.format(p, points[p]))
print('Data is requested at simulation time {0}'.format(time))

Coordinates of 3 points where variables are requested:
0: [11.349172   -0.34695953  6.1366253 ]
1: [ 5.752386   -0.56792283  9.062306  ]
2: [15.818903    0.11588102  6.1939235 ]
Data is requested at simulation time 7.457165950324574


In [17]:
print('Requesting velocity at {0} points...'.format(N))
result = lTDB.getData( time, points, data_set = 'channel',
        sinterp = spatialInterp, tinterp = temporalInterp,
        getFunction = 'Velocity' )
for p in range(N):
    print('{0}: {1}'.format(p, result[p]))

Requesting velocity at 3 points...
0: [ 1.0550174  -0.01989732 -0.06109373]
1: [0.9732515  0.03123313 0.01956358]
2: [ 1.10531     0.00639514 -0.01080191]


In [ ]:
print('Requesting pressure at {0} points...'.format(N))
result = lTDB.getData(time, points,data_set = 'channel',
        sinterp = spatialInterp, tinterp = temporalInterp,
        getFunction = 'getPressure')
for p in range(N):
    print('{0}: {1}'.format(p, result[p]))

In [ ]:
print('Requesting velocity and pressure at {0} points...'.format(N))
result = lTDB.getData(time, points,data_set = 'channel',
        sinterp = spatialInterp, tinterp = temporalInterp,
        getFunction = 'getVelocityAndPressure')
for p in range(N):
    print('{0}: {1}'.format(p, result[p]))

In [ ]:
print('Requesting velocity gradient at {0} points...'.format(N))
result = lTDB.getData(time, points,
        sinterp = FD4Lag4, tinterp = temporalInterp,data_set ='channel',
        getFunction = 'getVelocityGradient')
for p in range(N):
    print('{0}: '.format(p) +
          'duxdx = {0:+e}, duxdy = {1:+e}, duxdz = {2:+e}\n   '.format(result[p][0], result[p][1], result[p][2]) +
          'duydx = {0:+e}, duydy = {1:+e}, duydz = {2:+e}\n   '.format(result[p][3], result[p][4], result[p][5]) +
          'duzdx = {0:+e}, duzdy = {1:+e}, duzdz = {2:+e}'.format(result[p][6], result[p][7], result[p][8]))

In [ ]:
print('Requesting velocity hessian at {0} points...'.format(N))
result = lTDB.getData(time, points,data_set = 'channel',sinterp = FD4Lag4, tinterp = temporalInterp,
                      getFunction = 'getVelocityHessian')

for p in range(N):
        print('{0}: '.format(p) +
              'd2uxdxdx = {0:+e}, d2uxdxdy = {1:+e}, d2uxdxdz = {2:+e}\n   '.format(result[p][ 0], result[p][ 1], result[p][ 2])
            + 'd2uxdydy = {0:+e}, d2uxdydz = {1:+e}, d2uxdzdz = {2:+e}\n   '.format(result[p][ 3], result[p][ 4], result[p][ 5])
            + 'd2uydxdx = {0:+e}, d2uydxdy = {1:+e}, d2uydxdz = {2:+e}\n   '.format(result[p][ 6], result[p][ 7], result[p][ 8])
            + 'd2uydydy = {0:+e}, d2uydydz = {1:+e}, d2uydzdz = {2:+e}\n   '.format(result[p][ 9], result[p][10], result[p][11])
            + 'd2uzdxdx = {0:+e}, d2uzdxdy = {1:+e}, d2uzdxdz = {2:+e}\n   '.format(result[p][12], result[p][13], result[p][14])
            + 'd2uzdydy = {0:+e}, d2uzdydz = {1:+e}, d2uzdzdz = {2:+e}'.format(result[p][15], result[p][16], result[p][17]))
        

In [ ]:
print('Requesting velocity laplacian at {0} points...'.format(N))
result = lTDB.getData(time, points,
        sinterp = FD4Lag4, tinterp = temporalInterp, data_set = 'channel',
        getFunction = 'getVelocityLaplacian')
for p in range(N):
    print('{0}: '.format(p) +
          'grad2ux = {0:+e}, grad2uy = {1:+e}, grad2uz = {2:+e}, '.format(result[p][0], result[p][1], result[p][2]))

In [ ]:
print('Requesting pressure gradient at {0} points...'.format(N))
result = lTDB.getData(time, points,
        sinterp = FD4Lag4, tinterp = temporalInterp, data_set = 'channel',
        getFunction = 'getPressureGradient')
for p in range(N):
    print('{0}: '.format(p)
        + 'dpdx = {0:+e}, dpdy = {1:+e}, dpdz = {2:+e}, '.format(result[p][0], result[p][1], result[p][2]))

In [ ]:
print('Requesting pressure hessian at {0} points...'.format(N))
result = lTDB.getData(time, points,
        sinterp = FD4Lag4, tinterp = temporalInterp, data_set = 'channel',
        getFunction = 'getPressureHessian')
for p in range(N):
    print('{0}: '.format(p) +
          'd2pdxdx = {0:+e}, d2pdxdy = {1:+e}, d2pdxdz = {2:+e}\n   '.format(result[p][0], result[p][1], result[p][2])
        + 'd2pdydy = {0:+e}, d2pdydz = {1:+e}, d2pdzdz = {2:+e}'.format(result[p][3], result[p][4], result[p][5]))

In [ ]:
result = lTDB.getThreshold(
            data_set = 'channel',
            field = 'vorticity',
            time = 0,
            threshold = 0.0,
            cx = 0, cy = 0, cz = 0,
            nx = 4, ny = 4, nz = 4,
            sinterp = 40,
            tinterp = 0)
for p in range(N):
    print('{0}: '.format(p)
        + 'i_x = {0}, i_y = {1}, i_z = {2}, value = {3} '.format(result[p][0], result[p][1], result[p][2],result[p][3]))

In [ ]:
result = lTDB.getRawData(
            time,
            start = np.array([0, 0, 0], dtype = np.int),
            size  = np.array([8, 8, 8], dtype = np.int),
            data_set = 'channel',
            getFunction = 'Pressure')
print(result)

In [ ]:
x, t = lTDB.getPosition(
    starttime = 0.1,
    endtime = 0.2,
    dt = 0.01,
    data_set = 'channel',
    point_coords = points[0:1,:],
    steps_to_keep = 10)
print(x)

In [ ]:
t1 = np.linspace(0, 4*3.14, 64)
t2 = np.linspace(-1, 1, 64)
x = np.zeros((t1.shape[0], t2.shape[0], 3), np.float32)
x[:, :, 0] = t1[np.newaxis, :]
x[:, :, 1] = t2[:, np.newaxis]
x[:, :, 2] = .0

In [ ]:
T = pyJHTDB.dbinfo.channel['time'][-1]
time = np.random.random()*T
u = lTDB.getData(
               time,
               x,
               data_set = 'channel',
               sinterp = 4,
               getFunction='getVelocity')

In [ ]:
e = np.sum(u**2, axis = 2)
fig = plt.figure(figsize = (10, 5))
a = fig.add_subplot(121)
a.set_axis_off()
a.imshow(e,
         extent = [t1[0], t1[-1] - t1[0], t2[0], t2[-1] - t2[0]],
         interpolation = 'none')

In [ ]:
lTDB.finalize()